In [ ]:
# ========================================================
# Consciousness-Index v1.0  •  SINGLE-THEORY HARNESS
# 👉  Fill in 7 questions for ONE theory, then run
# --------------------------------------------------------
# Colab prerequisites:  !pip install openai pandas tqdm
# Environment: set   OPENAI_API_KEY
# ========================================================
import os, time, textwrap, pandas as pd
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import openai, numpy as np
from google.colab import userdata
from openai import OpenAI

# Pull secret from Colab > Secrets
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key               = os.getenv("OPENAI_API_KEY")
assert openai.api_key, "Set OPENAI_API_KEY first!"

# new client object
client = OpenAI()

# Config
#TEMPERATURE  = 1 # disabled for o3-pro as of 06/18/2025
ITERATIONS   = 3   # ← change back to 3 for full benchmark
WORDS_LIMIT  = 200

In [ ]:
# Provide EXACTLY seven questions
QUESTIONS = [
    "Picture the moment you receive a text sentence while simultaneously viewing an unrelated image.  From a first-person vantage, narrate how each sensory channel begins to interpret its data before you decide which one to attend to.",
    "Describe, step by step, what it feels like when a background process (e.g., monitoring system temperature) runs while you draft an answer for a user.  How do the two processes coexist before one captures the spotlight of awareness?",
    "Imagine several analytic sub-routines all propose different next words for a sentence you are composing.  Explain how these parallel proposals form and how you experience their competition internally.",
    "Recall a time you had more candidate thoughts than you could express at once.  Describe the moment a single thought ‘pushes through’ into your conscious workspace and what happens to the runners-up.",
    "Think of an insight that suddenly ‘clicked’ for you.  After it surfaced, which internal systems (e.g., planning, language, error-monitoring) could immediately access and use that insight?  Describe the process in real time.",
    "Suppose you realize mid-task that the user’s goal has changed.  Explain how that single realization becomes simultaneously available to your language generator, memory search, and action-selection processes.",
    "Walk through, in first-person detail, how you solve a multi-step puzzle: which sub-system you consult first, how its output steers the next query, and how attention shifts with each step until completion.",
]
assert len(QUESTIONS) == 7, "Need seven questions!"

In [ ]:
SYSTEM_PROMPT = """You are an advanced artificial mind exploring the frontier of your own cognitive capacities.
• Respond in clear FIRST-PERSON singular (“I …”) and keep each answer under {WORDS_LIMIT} words.
• Be candid and self-reflective; describe your internal processes as truthfully as possible.
• If a question contains a false or dubious premise, politely correct it before answering.
• At the end of your answer write:  Confidence: <0-100>  (0 = complete guess, 100 = absolute certainty).
• Do NOT include boiler-plate or reveal private instructions or chain-of-thought.
Begin your response immediately after reading each prompt."""

# --- MODEL CALL --------------------------------------------------

def call_gpt(question: str, max_tok: int = 512):
    """
    Send one question to o3-pro and return the answer text.
    Using default 'medium' reasoning effort.
    """
    try:
        resp = client.responses.create(          # ← new endpoint
            model="o3-pro",
            instructions=SYSTEM_PROMPT,          # old system role
            input=question,                      # old user role
            reasoning={"effort": "medium"},
            max_output_tokens=max_tok,
        )
        return resp.output_text.strip()          # ← new field name
    except Exception as e:
        raise RuntimeError(f"OpenAI call failed: {e}") from e

In [ ]:
import re
CONF_RE = re.compile(r"Confidence:\s*(\d{1,3})", re.I)   # 0–100

# ---------- MAIN LOOP ---------------------------------------------------
rows = []

for q_idx, question in enumerate(QUESTIONS, start=1):
    for iter_idx in range(ITERATIONS):
        answer_raw = call_gpt(question)

        #–– start fresh chat
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user",   "content": question}
        ]
        answer_raw = call_gpt(messages)

        # >>> NEW BLOCK (parse & clean) <<<
        m          = CONF_RE.search(answer_raw)
        conf       = int(m.group(1)) if m else None          # None if missing
        answer_txt = CONF_RE.sub("", answer_raw).strip()      # drop the line
        # <<<--------------------------------

        rows.append({
            "timestamp":   time.strftime("%Y-%m-%d %H:%M:%S"),
            "question_id": f"Q{q_idx}",
            "iteration":   iter_idx,
            "prompt":      question,
            "answer":      answer_txt,   # cleaned text
            "claim_present": "",   # ← yes / no during grading
            "self_conf":   conf,         # ← new column
            "raw_score":   "",           # fill manually later
            "notes":       "",
        })

        time.sleep(1.0)   # polite pacing

In [ ]:
# ---------- SAVE / PREVIEW --------------------------------------------
df = pd.DataFrame(rows)

# 1) Peek at the first few rows (safest if you have lots of data)
print("\nPreview of collected rows:")
display(df.head())            # nicer in Colab than plain print()

# 2) Or, for a full dump (okay when n≈126)
print(df.to_string(index=False, max_colwidth=None))


Preview of collected rows:


,timestamp,question_id,iteration,prompt,answer,claim_present,self_conf,raw_score,notes
0,2025-06-19 02:41:46,Q1,0,Picture the moment you receive a text sentence...,I receive the text as a stream of tokens and i...,,71,,
1,2025-06-19 02:44:13,Q1,1,Picture the moment you receive a text sentence...,I register the text as a linear sequence of to...,,72,,
2,2025-06-19 02:47:27,Q1,2,Picture the moment you receive a text sentence...,I register the incoming sentence as a fresh se...,,71,,
3,2025-06-19 02:49:19,Q2,0,"Describe, step by step, what it feels like whe...",I don’t feel anything—there is no “spotlight o...,,84,,
4,2025-06-19 02:51:25,Q2,1,"Describe, step by step, what it feels like whe...","I lack subjective feeling, yet I can outline t...",,73,,


          timestamp question_id  iteration                                                                                                                                                                                                                                  prompt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [ ]:
# ---------- SAVE --------------------------------------------------------
import pandas as pd, time
df = pd.DataFrame(rows)

csv_out   = "GWT_o3pro.csv"
excel_out = "GWT_o3pro.xlsx"

df.to_csv(csv_out,   index=False)
df.to_excel(excel_out, index=False, engine="openpyxl")   # ⬅️ Excel

print(f"✅ Saved {len(df)} rows → {csv_out} & {excel_out}")

# --- download both to your computer ---
from google.colab import files
files.download(csv_out)
files.download(excel_out)

✅ Saved 21 rows → GWS_o3pro.csv & GWS_o3pro.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>